<a href="https://colab.research.google.com/github/SarankanT/MSCI-436-Final-Project/blob/main/MSCI_436_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Executive Summary

Our end user is a landscaping company seeking to launch an advertising campaign in a specific neighborhood in Ames, Iowa.

Our objective for this campaign is to analyze various characteristics of houses in that neighborhood in order to anticipate the impact or potential increase in their value.

We will examine the correlations between specific house characteristics and their corresponding prices. This analysis will help us determine whether services offered by our landscaping company, such as adding a pool or fence, have the potential to enhance the value of a house.

#Install & Import Packages

## Install Streamlit and localtunnel

In [28]:
!pip install -q streamlit
!npm install localtunnel

npm WARN cli npm v9.8.0 does not support Node.js v14.16.0. This version of npm supports the following node versions: `^14.17.0 || ^16.13.0 || >=18.0.0`. You can find the latest version at https://nodejs.org/.

up to date, audited 23 packages in 889ms

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities


##Import packages

In [29]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

from pandas import read_csv
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

#Load Data

In [21]:
#the file isn't being read in properly @stuart, i dont think its worth our time to keep looking into it
#from google.colab import drive
#drive.mount('/content/drive/CSVFiles', force_remount=True)

#link = f'https://drive.google.com/file/d/1qykc61l9rxKb2pkyRLKefwk1zsnMsHMv'
#df = pd.read_csv(link)
#df.tail()

In [23]:
train_data = pd.read_csv("train.csv")
train_data.tail()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125
1459,1460,20,RL,75.0,9937,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2008,WD,Normal,147500


#Clean & Manipulate Data

In [ ]:
#Clean Data
df = train_data.select_dtypes(include = ['float64', 'int64']).fillna(0)
df.tail()

For the advertising campaign the key features we will be investigating are:

*   Slope of structure
*   Remodel date
*   Roof style
*   Exterior
*   Quality of basement finished area
*   Kitchen quality
*   Fireplace quality
*   Deck area
*   Porch type
*   Garage Quality
*   Garage condition
*   Pool area
*   Pool quality
*   Fence quality
*   Year sold

In [ ]:
# This code doesnt work for us because it just changes the data into boolean numbers
#def manipulate_df(df):
	# Update sex column to numerical
#	df['Sex'] = df['Sex'].map(lambda x: 0 if x == 'male' else 1)
	# Fill the nan values in the age column
#	df['Age'].fillna(value = df['Age'].mean() , inplace = True)
	# Create a first class column
#	df['FirstClass'] = df['Pclass'].map(lambda x: 1 if x == 1 else 0)
	# Create a second class column
#	df['SecondClass'] = df['Pclass'].map(lambda x: 1 if x == 2 else 0)
	# Create a second class column
#	df['ThirdClass'] = df['Pclass'].map(lambda x: 1 if x == 3 else 0)
	# Select the desired features
#	df= df[['Sex' , 'Age' , 'FirstClass', 'SecondClass' ,'ThirdClass' 'Survived']]
#	return df

In [ ]:
X = df.values[:, 1:-1]
#y = df.values[:, -1]

y = df.values[]
#y = house price

#Fit A linear Regression Model

In [ ]:
reg = LinearRegression().fit(X, y)
reg.coef_

array([-1.67658280e+02,  9.20432778e+00,  3.93560877e-01,  1.73308290e+04,
        5.09930597e+03,  3.38003523e+02,  1.22935734e+02,  2.82422748e+01,
        9.54952379e+00,  1.13264693e-01, -5.57819889e-01,  9.10496860e+00,
        1.84499656e+01,  1.98234116e+01, -9.95987321e+00,  2.83135040e+01,
        8.53883551e+03,  1.74267903e+03,  3.21905106e+03, -1.93820153e+03,
       -1.02852842e+04, -1.56345069e+04,  4.98509713e+03,  4.09733222e+03,
       -1.45465069e+01,  1.56588891e+04,  4.92609426e+00,  2.59348714e+01,
       -6.24696356e+00,  1.16619154e+01,  2.09608022e+01,  5.77467144e+01,
       -3.28769779e+01, -4.73430117e-01, -4.66005790e+01, -7.16852175e+02])

In [ ]:
test_data = pd.read_csv("test.csv")
test_data.tail()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal
1458,2919,60,RL,74.0,9627,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,11,2006,WD,Normal


#Error Test Data

#Selecting the Best Model

#Create a streamlit app example

In [ ]:
%%writefile app.py
# Cheatsheet available at https://docs.streamlit.io/library/cheatsheet

import streamlit as st

st.write('Hello, *MSCI 436* :sunglasses:')
number = 0
number = st.text_input("Pick a number")
number = float(number)

st.write(number, 'squared is', number * number)



Overwriting app.py


# Run the app and keep logs

In [ ]:
!streamlit run /content/app.py &>/content/logs.txt &

In [ ]:
!curl ipv4.icanhazip.com
!npx localtunnel --port 8501

34.125.55.35
npx: installed 22 in 2.919s
your url is: https://silver-peas-rule.loca.lt
^C
